<a href="https://colab.research.google.com/github/elango-ela/Twitter-Agent-for-Posting-Crypto-News/blob/main/Twitter_Agent_for_Posting_Cryto_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tweepy

In [ ]:
import requests
import tweepy
import google.generativeai as genai
from bs4 import BeautifulSoup

# Twitter API credentials (replace with your actual keys)
BEARER_TOKEN = "YOUR_BEARER_TOKEN"
CONSUMER_KEY = "YOUR_CONSUMER_KEY"
CONSUMER_SECRET = "YOUR_CONSUMER_SECRET"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"
ACCESS_TOKEN_SECRET = "YOUR_ACCESS_TOKEN_SECRET"

# Google Gemini API Key (replace with your actual key)
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"
genai.configure(api_key=GEMINI_API_KEY)

# ------------------------------------------
# 1️⃣ FETCH CRYPTO NEWS FROM CRYPTOCOMPARE
# ------------------------------------------
def fetch_crypto_news():
    url = "https://min-api.cryptocompare.com/data/v2/news/?lang=EN"
    response = requests.get(url)

    if response.status_code == 200:
        news_data = response.json()
        articles = news_data.get("Data", [])[:5]  # Limit to first 5 articles
        news_list = []

        for article in articles:
            title = article["title"]
            article_url = article["url"]

            print(f"\nFetching article: {title}")
            content = extract_article_content(article_url)

            news_list.append({"title": title, "content": content})

        return news_list
    else:
        print("Failed to fetch crypto news. Status Code:", response.status_code)
        return []

# ------------------------------------------
# 2️⃣ EXTRACT ARTICLE CONTENT
# ------------------------------------------
def extract_article_content(url):
    """Fetches and extracts article content from the given URL."""
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            paragraphs = soup.find_all("p")
            return "\n".join(p.get_text() for p in paragraphs[:5])  # Extract first 5 paragraphs
        else:
            return f"Failed to fetch article. Status Code: {response.status_code}"

    except Exception as e:
        return f"Error extracting content: {str(e)}"

# ------------------------------------------
# 3️⃣ SUMMARIZE NEWS USING GEMINI AI
# ------------------------------------------
def summarize_news(news_list):
    """Uses Google Gemini AI to summarize crypto news into a tweetable format."""
    news_text = "\n\n".join([f"Title: {news['title']}\nContent: {news['content']}" for news in news_list])

    prompt = f"""Pick the most important cryptocurrency news and summarize it in a single paragraph without any full stops, emojis, or hashtags. Make it engaging and easy to read. Note it should be only 280 characters precisely so don't skip any line to next line.

    News:\n{news_text}"""

    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)

    return response.text.strip() if response.text else "Crypto markets are moving fast! Stay updated."

# ------------------------------------------
# 4️⃣ POST TWEET VIA TWEEPY
# ------------------------------------------
def post_tweet(tweet_text):
    """Posts the summarized crypto news to Twitter."""
    client = tweepy.Client(
        bearer_token=BEARER_TOKEN,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        access_token=ACCESS_TOKEN,
        access_token_secret=ACCESS_TOKEN_SECRET,
        wait_on_rate_limit=True
    )

    response = client.create_tweet(text=tweet_text)
    print(f"Tweet posted: https://twitter.com/user/status/{response.data['id']}")

# ------------------------------------------
# 5️⃣ MAIN FUNCTION TO EXECUTE ALL STEPS
# ------------------------------------------
def main():
    articles = fetch_crypto_news()  # Step 1: Fetch crypto news

    if articles:
        tweet_text = summarize_news(articles)  # Step 2: Summarize news
        print(f"Generated Tweet: {tweet_text}")

        post_tweet(tweet_text)  # Step 3: Post to Twitter

if __name__ == "__main__":
    main()


Fetching article: Bitcoin (BTC) Price Prediction for March 19

Fetching article: Race to List: Exchanges Are Competing for Adding New Tokens Faster

Fetching article: Michael Saylor Stuns With BTC Statement as Bitcoin Returns to Green

Fetching article: MicroStrategy’s Bitcoin Gamble: Will Debt and Stock Dilution Lead to Collapse?

Fetching article: Binance Founder CZ Changed His Profile Photo, This Altcoin Price First Pumped, Then Dumped!
Generated Tweet: Bitcoin price eyes $85000 if buyers keep the pressure while Michael Saylor suggests Bitcoin as a hedge against currency devaluation like the Turkish lira plus a memecoin linked to Binance founder CZ saw a price surge after CZ updated his profile.
Tweet posted: https://twitter.com/user/status/1902374592401822095
